# Module 2 Assessment

Welcome to your Mod 2 Assessment. You will be tested for your understanding of concepts and ability to solve problems that have been covered in class and in the curriculum.

Use any libraries you want to solve the problems in the assessment.

You will have up to two hours to complete this assessment.

The sections of the assessment are:

- SQL and Relational Databases
- Object Oriented Programming
- Accessing Data Through APIs
- HTML, CSS and Web Scraping
- Other Database Structures (MongoDB)

In [1]:
# import the necessary libraries

import requests
import json
import pandas as pd
import sqlite3

## Pokemon

### Section 1: Getting Data

In this section we'll be using PokeAPI to get data on Pokemon. Let's first define functions to get information from the API. Provided below is a URL that will get you started with the first 151 Pokemon! Run the cell below to see what we get.

In [2]:
url = 'https://pokeapi.co/api/v2/pokemon/?limit=151'
results = requests.get(url).json()['results']
results

[{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
 {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
 {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
 {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
 {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
 {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
 {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
 {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
 {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
 {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
 {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
 {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
 {'name': 'weedle', 'url': 'https://pokeapi.co/api/v2/pokemon/13/'},
 {'name': 'kakuna', 'url': 'https://pokeapi.co/api/v2/pokemon/14/'},
 {'name': '

[Read the documentation here](https://pokeapi.co/) for information on navigating this API and use the API to obtain data to answer the following questions.

1. For any **one** Pokemon, retrive the following information in a dictionary format with the following keys:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities

For `Types` and `Abilities`, you might want to write helper functions to have each attribute just be a list of types and a list of abilities. Your output should look like this:

```
{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}

```
    
    
2. Get the same information for the first **151** Pokemon as a list of dictionaries ordered by Pokemon ID. Print the first and last elements of the dictionary. (Hint: Use pagination) Your output should save the list to a variable and look like this:

```
[{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, 
{'id': 2, 
'name': 'ivysaur', 
'base_experience': 142, 
'weight': 130, 
'height': 10, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, ... ]

```



In [3]:
""" SOLUTION: data for one Pokemon """

# helper functions for types and abilities

def typelist(types):
    result = []
    
    # iterating through the nested dict and appending to the empty list:
    
    for i in range(len(types)):
        result.append(types[i]['type']['name'])
    return result

def abilitylist(abilities):
    result = []
    
    # iterating through the nested dict and appending to the empty list:
    
    for i in range(len(abilities)):
        result.append(abilities[i]['ability']['name'])
    return result

In [4]:
def get_pokedata(url):
    
    # getting full results for one pokemon
    info = requests.get(url).json() 
    
    # list of keys with values that don't need editing
    keys = ['id', 'name', 'base_experience', 'weight', 'height'] 
    data = {k: info[k] for k in keys} # dictionary comprehension
    
    # using the two helper functions to add types and abilities
    data['types'] = typelist(info['types'])
    data['abilities'] = abilitylist(info['abilities'])
    
    return data

In [5]:
""" SOLUTION: data for 151 Pokemon """

# list comprehension to get a list of just URLs
urls = [r['url'] for r in results]

# list comprehension with the previous function to get full data
pokedata = [get_pokedata(url) for url in urls]


In [6]:
# printing first and last elements

print(pokedata[0], pokedata[-1])

{'id': 1, 'name': 'bulbasaur', 'base_experience': 64, 'weight': 69, 'height': 7, 'types': ['poison', 'grass'], 'abilities': ['chlorophyll', 'overgrow']} {'id': 151, 'name': 'mew', 'base_experience': 270, 'weight': 40, 'height': 4, 'types': ['psychic'], 'abilities': ['synchronize']}


### Section 2: Object Oriented Programming

We're going to use the data gathered in the previous section on APIs for this section on Object Oriented Programming to instantiate Pokemon objects and write instance methods.

#### Creating a Class

1. Create a class called `Pokemon` with an `__init__` method to instantiate the following attributes:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities
    
    
#### Instantiating Objects

2. Using the data you obtained from the API, instantiate the first, fourth and seventh Pokemon. Assign them to the variables `bulbasaur`, `charmander` and `squirtle`.


In [ ]:
# if you were unable to get the data from the API in the right format,
# uncomment the code below to access a JSON file with the list of dictionaries

# with open('data/pokemon.json') as f:  
#     pokelist = json.load(f)

In [7]:
class Pokemon:
    
    def __init__(self, ID, name, exp, weight, height, types, abilities):
        self.ID = ID
        self.name = name
        self.exp = exp
        self.weight = weight
        self.height = height
        self.types = types
        self.abilities = abilities
        
    def bmi(self):
        return (self.weight*0.1)/(self.height*0.1)**2
        


In [11]:
""" SOLUTION: instantiating three Pokemon """

# function to instantiate a Pokemon
def instantiate_pokemon(info):
    pokemon = Pokemon(info['id'], 
                      info['name'], 
                      info['base_experience'], 
                      info['weight'], 
                      info['height'], 
                      info['types'],
                      info['abilities'])
    return pokemon

"""
can also be done manually:

bulbasaur = Pokemon(1, 'bulbasaur', 64, 69, 7, ['poison', 'grass'], ['chlorophyll', 'overgrow'])

etc.

"""

bulbasaur = instantiate_pokemon(pokedata[0])
charmander = instantiate_pokemon(pokedata[3])
squirtle = instantiate_pokemon(pokedata[6])

In [12]:
# run this cell to test and check your code
# you may need to edit the attribute variable names if you named them differently!

def print_pokeinfo(pokemon_object):
    o = pokemon_object
    print('ID: ' + str(o.ID) + '\n' +
          'Name: ' + o.name.title() + '\n' +
          'Base experience: ' + str(o.exp) + '\n' +
          'Weight: ' + str(o.weight) + '\n' +
          'Height: ' + str(o.height) + '\n' +
          'Types: ' + str(o.types) + '\n' +
          'Abilities: ' + str(o.abilities) + '\n'
         )
    
print_pokeinfo(bulbasaur)
print_pokeinfo(ivysaur)
print_pokeinfo(venusaur)

ID: 1
Name: Bulbasaur
Base experience: 64
Weight: 69
Height: 7
Types: ['poison', 'grass']
Abilities: ['chlorophyll', 'overgrow']

ID: 4
Name: Charmander
Base experience: 62
Weight: 85
Height: 6
Types: ['fire']
Abilities: ['solar-power', 'blaze']

ID: 7
Name: Squirtle
Base experience: 63
Weight: 90
Height: 5
Types: ['water']
Abilities: ['rain-dish', 'torrent']



#### Instance Methods

3. Write an instance method within the class `Pokemon` to find the BMI of a Pokemon. BMI is defined by $\frac{weight}{height^{2}}$ with weight in **kilograms** and height in **meters**. The height and weight data of Pokemon from the API is in **decimeters** and **hectograms** respectively.


    1 decimeter = 0.1 meters
    1 hectogram = 0.1 kilograms

In [13]:
# run this cell to test and check your code
# you probably have to rerun the code to instantiate your objects

print(bulbasaur.bmi()) # 14.08
print(charmander.bmi()) # 23.61
print(squirtle.bmi()) # 36

14.081632653061222
23.611111111111104
36.0


## SQL Queries

For this section, we've put the Pokemon data into SQL tables. You won't need to use your list of dictionaries or the JSON file for this section. The schema of `pokemon.db` is as follows:

<img src="data/pokemondb.png" alt="db schema" style="width:500px;"/>

Assign your SQL queries as strings to the variables `q1`, `q2`, etc. and run the cells at the end of this section to print your results as Pandas DataFrames.

- q1: query all columns from `Pokemon` the Pokemon that have base_experience above 200
- q2: query the id, name, type1 and type2 of Pokemon that have **water** types as either their first or second type
- q3: query the average weight of Pokemon by their first type in descending order
- q4: query the Pokemon name, Pokemon type2, and what **type2** has "2xdamage" to
- q5: query the top 5 most common type1s, the minimum height, maximum height, minimum weight and maximum weight of pokemon with those type1s, and what associated type they do "0.5xdamage" to


**Important note on syntax**: use `double quotes ""` when quoting strings **within** your query and wrap the entire query in `single quotes ''` For the column titles that begin with numbers, you need to wrap the column names in double quotes.

In [15]:
cnx = sqlite3.connect('data/pokemon.db')

In [17]:
q1 = 'select * from pokemon where base_experience > 150'
pd.read_sql(q1, cnx)

,id,name,base_experience,weight,height,type1,type2
0,3,venusaur,236,1000,20,grass,poison
1,6,charizard,240,905,17,fire,flying
2,9,blastoise,239,855,16,water,None
3,12,butterfree,178,320,11,bug,flying
4,15,beedrill,178,295,10,bug,poison
5,18,pidgeot,216,395,15,normal,flying
6,22,fearow,155,380,12,normal,flying
7,24,arbok,157,650,35,poison,None
8,26,raichu,218,300,8,electric,None
9,28,sandslash,158,295,10,ground,None


In [18]:
q2 = 'select id, name, type1, type2 from pokemon where type1 == "water" or type2 == "water"'
pd.read_sql(q2, cnx)

,id,name,type1,type2
0,7,squirtle,water,None
1,8,wartortle,water,None
2,9,blastoise,water,None
3,54,psyduck,water,None
4,55,golduck,water,None
5,60,poliwag,water,None
6,61,poliwhirl,water,None
7,62,poliwrath,water,fighting
8,72,tentacool,water,poison
9,73,tentacruel,water,poison


In [19]:
q3 = 'select type1, avg(weight) from pokemon group by type1 order by avg(weight) desc'
pd.read_sql(q3, cnx)

,type1,avg(weight)
0,rock,876.111111
1,dragon,766.000000
2,water,579.678571
3,fighting,542.857143
4,psychic,515.625000
5,normal,500.863636
6,fire,480.250000
7,ice,480.000000
8,ground,452.625000
9,electric,317.888889


In [20]:
q4 = 'select pokemon.name, type2, "2xdamage" from pokemon join types on pokemon.type2 = types.name'
pd.read_sql(q4, cnx)

,name,type2,2xdamage
0,bulbasaur,poison,grass
1,ivysaur,poison,grass
2,venusaur,poison,grass
3,charizard,flying,fighting
4,butterfree,flying,fighting
5,weedle,poison,grass
6,kakuna,poison,grass
7,beedrill,poison,grass
8,pidgey,flying,fighting
9,pidgeotto,flying,fighting


In [21]:
q5 = 'select type1, count(*), min(height), max(height), min(weight), max(weight), "0.5xdamage" from pokemon \
        join types on pokemon.type1 = types.name \
        group by type1 \
        order by count(*) desc limit 5'
pd.read_sql(q5, cnx)

,type1,count(*),min(height),max(height),min(weight),max(weight),0.5xdamage
0,water,28,3,65,40,2350,steel
1,normal,22,3,22,18,4600,None
2,poison,14,4,35,10,650,fighting
3,bug,12,3,15,29,560,fighting
4,fire,12,6,20,85,1550,bug
